In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as px


In [3]:
df = pd.read_csv('data/08_Data.csv')
print(df.shape)
df.head(2)

(93043, 17)


,Unnamed: 0,Code_Num,Datetime,Process_Type,ST,Tube Furnace CP,Tube Furnace1 OP,Tube Furnace1 Temp,Tube Furnace2 OP,Tube Furnace2 Temp,Tube Furnace3 OP,Tube Furnace3 Temp,Tube Furnace4 OP,Tube Furnace4 Temp,Z1-OP1,Z1-OP2,Z1-Temp
0,0,GroupA,2022-01-03 11:22,OP-A,NaN,0.450497,73.527404,NaN,59.989422,868.759544,52.249481,NaN,72.134908,NaN,74.255730,76.879748,NaN
1,1,GroupA,2022-01-03 11:22,OP-A,NaN,0.457370,79.149174,878.116929,62.027232,883.821264,50.617109,890.74076,71.245235,894.430606,66.323571,71.297720,NaN


- 통계적 가설 검정
1. X 범주형, Y 연속형 : 집단 간 대표값 비교
   - 정규성 검정: stats.normaltest()
   - 등분산 검정: stats.bartelett()
   - Y 정규 분포
     - 등분산: 2집단(stats.ttest_ind()), 3집단 (stats.f_oneway())
     - 이분산: 2집단(stats.ttest_ind()), 3집단 (stats.kruskal())
   - Y 비모수 분포
     - 2집단: stats.ranksums(), 3집단 (stats.kruskal())
    
2. X 연속형, Y 연속형: 두 숫자 데이터의 상관성 검정
   - Y 정규분포: stats.pearsonr()
   - Y 비모수: stats.spearmanr()
3. X 범주형, Y 범주형: 두 범주형 항목이 서로 연관성이 있는지 검정
   - stats.chi2_contingency()

In [8]:
# process_type에 따라서, code_num의 비율이 달라지는가?
p1 = pd.crosstab(df['Process_Type'], df['Code_Num'])
c1 = p1.iloc[1:][['GroupA', 'GroupB', 'GroupC']]

In [12]:
# 귀무가설: process_type과 conde_num은 서로 독립이다.
# 대립가설: process_type과 conde_num은 서로 독립이 아니다.

stats.chi2_contingency(c1)
# 귀무가설 기각 -> 서로 독립이 아니다.

Chi2ContingencyResult(statistic=np.float64(8748.092103980418), pvalue=np.float64(0.0), dof=2, expected_freq=array([[ 2826.81681172,  5504.22207599, 11166.96111229],
       [ 4003.18318828,  7794.77792401, 15814.03888771]]))

In [11]:
# GroupA와 GroupC 라인에 대해서 공정프로세스 OP-B와 OP-C의 비율의 유의미한 차이가 있는지?
# 귀무가설: GroupA와 GroupC라인에 대해 공정프로세스 OP-B와 OP-C는 독립이다.
# 대립가설: GroupA와 GroupC라인에 대해 공정프로세스 OP-B와 OP-C는 독립이 아니다.

stats.chi2_contingency(c1[['GroupA', 'GroupC']])
# 귀무가설 기각 -> 서로 독립이 아니다.

Chi2ContingencyResult(statistic=np.float64(339.71650764322374), pvalue=np.float64(7.355467265232657e-76), dof=1, expected_freq=array([[ 1932.58436603,  7634.41563397],
       [ 4897.41563397, 19346.58436603]]))

- 상관성 검정(연속형 - 연속형)과 독립성 검정(범주형 - 범주형)는 서로 연관이 있는지, 독립인지를 알 수 있을 뿐, 인과관계를 알수는 없다.

## 10-4. 인과 추론(Causal Inference)

- 데이터 간 단순한 상관관계 (Correlation)를 넘어선 '원인과 결과'의 관계를 밝힐 때 사용
- 어떤 원인(Cause)이 어떤 결과(effect)를 가져왔는지를 데이터를 통해 추론
  - 상관관계: '동물의 숲 노래 청취시간과 개별미션 점수의 상관관계가 있을까?'
  - 인과질문: '동물의 숲 노래 청취를 오래하면, 개별미션 점수는 올라갈까?'
- 인과 추론 도출 방법론:
  - 1. 무작위 대조 실험(RCT, Randomized Controlled Trial)
       - 실험군과 대조군을 무작위로 나누어 인과 효과를 명확히 측정
       - 매우 강력함, 비용과 시간, 윤리적 문제 발생
   
  - 2. 관측 데이터 기반 한 방법 (Observaional Data)
       - 실험이 제한되는 상황에서, 기존에 확보된 정보 및 데이터를 기반으로 인과관계를 측정
       - 부정확, 외부적인 개입
   
- 상관성 검정 이후에, 조금 더 강력한 근거를 찾기위한 정도로 사용
- 실제 분석에서는 인과추론 모델 설계 시, 도메인지식을 반드시 필요

In [13]:
# 인과 추론 라이브러리 Dowhy
# !pip install dowhy

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 13.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/989.5 kB ? eta -:--:--
   --------------------------------------- 989.5/989.5 kB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/742.5 kB ? eta -:--:--
   --------------------------------------- 742.5/742.5 kB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
   --------------------- ------------------ 3.9/7.4 MB 19.5 MB/s eta 0:00:01
   -------------------------------------- - 7.1/7.4 MB 17.4 MB/s eta 0:00:01
   ---------------------------------------- 7.4/7.4 MB 15.8 MB/s eta 0:00:00


In [14]:
# 라이브러리 호출
import dowhy
from dowhy import CausalModel

In [15]:
# Tube Furnace CP와 ST값 간의 상관성이 있는가?
# 정규성 검정
df2 = df[['ST', 'Tube Furnace CP']].dropna()
df2

,ST,Tube Furnace CP
2,68.724636,0.454914
3,69.161116,0.456540
4,69.181098,0.460100
5,69.472810,0.459051
6,69.189685,0.459515
...,...,...
93038,71.033373,0.452082
93039,70.279919,0.454568
93040,70.743509,0.449965
93041,70.135205,0.454477


In [16]:
# 귀무가설: 해당 데이터의 분포는 정규분포를 따른다.
# 대립가설: 해당 데이터의 분포는 정규분포를 따르지 않는다.

print(stats.normaltest(df2['ST']))
print(stats.normaltest(df2['Tube Furnace CP']))
# 귀무가설 기각 -> 정규분포를 따르지 않는다

NormaltestResult(statistic=np.float64(3441.4905138133176), pvalue=np.float64(0.0))
NormaltestResult(statistic=np.float64(46066.17234762602), pvalue=np.float64(0.0))


In [17]:
# 상관성 검정(비모수 데이터 - 정규분포를 따르지 않는 데이터)
# 귀무가설: 두 연속형 자료는 서로 상관성이 없다.
# 대립가설: 두 연속형 자료는 서로 상관성이 있다.
stats.spearmanr(df2['ST'], df2['Tube Furnace CP'])
# 귀무가설 기각 -> 서로 상관성이 있다.

SignificanceResult(statistic=np.float64(-0.008256910985147173), pvalue=np.float64(0.01178868290438504))

In [20]:
# 인과 추론 모델을 구성
model = CausalModel(df2, treatment = 'ST', outcome = 'Tube Furnace CP', common_causes = None)
# treatment: 원인변수 설정(원인 변수에 해당하는 데이터를 가정하여 입력)
# outcome: 결과변수 설정
# common_causes: 혼란변수, 두 변수(원인, 결과 변수)에 모두 영향을 줄 수 있는 변수

In [22]:
model_result = model.identify_effect()
print(model_result)
# Estimand type: 추정하고자 하는 인과 효과
# NONPARAMETRIC_ATE: 어떤 특정한 모델의 형태 (선형, 로그 등)를 가정하지 않고, 데이터를 통해 직접적으로 평균과 효과를 추정하는 방식
# ATE(average treatment effect): 전체 모집단에서 '평균적으로' 원인변수가 결과변수에 미치는 영향
# Estimand assumption 1, Unconfoundedness: 비혼란성(Unconfoundedness)을 가정
# CP값과 ST값 사이에 영향을 주는 공통원인(confounder)이 없거나, 이를 통제할 수 있다는 전제를 놓고 분석(이 가정이 성립되면 ATE가 신뢰성 있게 추정 가능)

Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                      
─────(E[Tube Furnace CP])
d[ST]                    
Estimand assumption 1, Unconfoundedness: If U→{ST} and U→Tube Furnace CP then P(Tube Furnace CP|ST,,U) = P(Tube Furnace CP|ST,)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



In [26]:
# Estimand name: iv No such variable(s) found!
# 해당 라이브러리가 도구변수(Instrumental Variable)로 사용할 수 있는 값을 찾지 못했다는 의미
# 도구 변수: 원인 변수에는 영향을 주지만, 결과변수에 직접적인 영향을 주지 않는 중간 조정 변수

# Estimand name: frontdoor No such variable(s) found!
# 중간 매개 변수(mediator)를 찾지 못했다는 의미
# 중간 매개 변수(mediator): 원인과 결과 사이에 발생하는 중간단계 데이터
# ST-> 중간단계 -> CP 이러한 구조를 찾지 못했다고 판단

# 도출된 인과 추론 모델을 모형화하여, 원인이 얼만큼 바뀔 때, 결과가 얼만큼 변하는지 계산
est = model.estimate_effect(model_result, method_name = 'backdoor.linear_regression')
est.value # ST값이 1 움직일 때 cp값은 -0.000297정도 변한다

C:\Users\leedo\anaconda3\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



np.float64(-0.00029733378235602403)

In [27]:
est.get_confidence_intervals() # 인과분석 결과의 신뢰구간
# 'ST값이 1씩 증가할 때, 평균적으로 CP값이 -0.0003524에서 -0.0002427로 감소할 것으로 95% 확신한다.'

array([[-0.0003524 , -0.00024227]])